# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [10]:
# import libraries
import pandas as pd
from sqlalchemy import create_engine
import re
import nltk
import string
import numpy as np

from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.multioutput import MultiOutputClassifier
from sklearn.pipeline import Pipeline

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.model_selection import GridSearchCV

from sklearn.metrics import accuracy_score, precision_score, recall_score

In [11]:
nltk.download(['punkt', 'wordnet', 'stopwords'])

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [43]:
nltk.download(['averaged_perceptron_tagger'])

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


True

In [12]:
# load data from database
engine = create_engine('sqlite:///DisasterResponse.db')
print(engine)
df = pd.read_sql_table('labeled_messages', engine)
X = df['message']
Y = df.drop(['message', 'genre', 'id', 'original'], axis=1) 

Engine(sqlite:///DisasterResponse.db)


### 2. Write a tokenization function to process your text data

In [13]:
stop_words = nltk.corpus.stopwords.words("english")
lemmatizer = nltk.stem.wordnet.WordNetLemmatizer()
remove_punc_table = str.maketrans('', '', string.punctuation)

In [49]:
def tokenize(text):
    # normalize case and remove punctuation
    text = text.translate(remove_punc_table).lower()
    
    # tokenize text
    tokens = nltk.word_tokenize(text)
    
    # lemmatize and remove stop words
    #return [lemmatizer.lemmatize(word) for word in tokens if word not in stop_words]
    return [lemmatizer.lemmatize(word).lower().strip() for word in tokens if word not in stop_words]


### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [15]:
forest_clf = RandomForestClassifier(n_estimators=10)
pipeline = Pipeline([
                    ('tfidf', TfidfVectorizer(tokenizer=tokenize)),
                    ('forest', MultiOutputClassifier(forest_clf))
                    ])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [16]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.33, random_state=42)

In [17]:
#X_train

In [18]:
pipeline.fit(X_train, Y_train)

Pipeline(memory=None,
         steps=[('tfidf',
                 TfidfVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.float64'>,
                                 encoding='utf-8', input='content',
                                 lowercase=True, max_df=1.0, max_features=None,
                                 min_df=1, ngram_range=(1, 1), norm='l2',
                                 preprocessor=None, smooth_idf=True,
                                 stop_words=None, strip_accents=None,
                                 sublinear_tf=False,
                                 token_pattern='...
                                                                        ccp_alpha=0.0,
                                                                        class_weight=None,
                                                                        criterion='gini',
                                  

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [19]:
Y_pred = pipeline.predict(X_test)

In [20]:
for i, col in enumerate(Y_test):
    print(col)
    print(classification_report(Y_test[col], Y_pred[:, i]))

related
              precision    recall  f1-score   support

         0.0       0.31      0.14      0.20      2096
         1.0       0.76      0.90      0.82      6497

    accuracy                           0.71      8593
   macro avg       0.54      0.52      0.51      8593
weighted avg       0.65      0.71      0.67      8593

request
              precision    recall  f1-score   support

         0.0       0.84      0.98      0.90      7117
         1.0       0.41      0.08      0.14      1476

    accuracy                           0.82      8593
   macro avg       0.62      0.53      0.52      8593
weighted avg       0.76      0.82      0.77      8593

offer
              precision    recall  f1-score   support

         0.0       0.99      1.00      1.00      8547
         1.0       0.00      0.00      0.00        46

    accuracy                           0.99      8593
   macro avg       0.50      0.50      0.50      8593
weighted avg       0.99      0.99      0.99      859

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

         0.0       0.89      0.99      0.94      7623
         1.0       0.20      0.01      0.02       970

    accuracy                           0.88      8593
   macro avg       0.54      0.50      0.48      8593
weighted avg       0.81      0.88      0.83      8593

shelter
              precision    recall  f1-score   support

         0.0       0.91      0.99      0.95      7833
         1.0       0.10      0.01      0.02       760

    accuracy                           0.91      8593
   macro avg       0.51      0.50      0.48      8593
weighted avg       0.84      0.91      0.87      8593

clothing
              precision    recall  f1-score   support

         0.0       0.98      1.00      0.99      8453
         1.0       0.00      0.00      0.00       140

    accuracy                           0.98      8593
   macro avg       0.49      0.50      0.50      8593
weighted avg       0.97      0.98      0.98      8593

mo

### 6. Improve your model
Use grid search to find better parameters. 

In [21]:
'''
parameters = {
    'tfidf__ngram_range': ((1, 1), (1, 2)),
    'tfidf__max_df': (0.8, 1.0),
    'tfidf__max_features': (None, 10000),
    'forest__estimator__n_estimators': [50, 100],
    'forest__estimator__min_samples_split': [2, 4]
}
'''

parameters = {
    'tfidf__ngram_range': ((1, 1), (1, 2))
}

cv = GridSearchCV(pipeline, parameters, cv=3, n_jobs=-1, verbose= 10)

In [22]:
cv.fit(X_train, Y_train)

Fitting 3 folds for each of 2 candidates, totalling 6 fits
[CV] tfidf__ngram_range=(1, 1) .......................................
[CV] ........... tfidf__ngram_range=(1, 1), score=0.139, total=  48.2s
[CV] tfidf__ngram_range=(1, 1) .......................................


[Parallel(n_jobs=-1)]: Done   1 out of   1 | elapsed:   48.2s remaining:    0.0s


[CV] ........... tfidf__ngram_range=(1, 1), score=0.149, total=  49.4s
[CV] tfidf__ngram_range=(1, 1) .......................................


[Parallel(n_jobs=-1)]: Done   2 out of   2 | elapsed:  1.6min remaining:    0.0s


[CV] ........... tfidf__ngram_range=(1, 1), score=0.141, total=  48.5s
[CV] tfidf__ngram_range=(1, 2) .......................................


...

[CV] ........... tfidf__ngram_range=(1, 2), score=0.138, total= 1.6min
[CV] tfidf__ngram_range=(1, 2) .......................................


[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:  5.5min remaining:    0.0s


[CV] ........... tfidf__ngram_range=(1, 2), score=0.132, total= 1.5min


[Parallel(n_jobs=-1)]: Done   6 out of   6 | elapsed:  7.0min remaining:    0.0s
[Parallel(n_jobs=-1)]: Done   6 out of   6 | elapsed:  7.0min finished


GridSearchCV(cv=3, error_score=nan,
             estimator=Pipeline(memory=None,
                                steps=[('tfidf',
                                        TfidfVectorizer(analyzer='word',
                                                        binary=False,
                                                        decode_error='strict',
                                                        dtype=<class 'numpy.float64'>,
                                                        encoding='utf-8',
                                                        input='content',
                                                        lowercase=True,
                                                        max_df=1.0,
                                                        max_features=None,
                                                        min_df=1,
                                                        ngram_range=(1, 1),
                                                        n

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [23]:
def evaluate_model(model, X_test, Y_test):
    Y_pred = model.predict(X_test)
    print(classification_report(Y_test, Y_pred, target_names=category_names))
#     print('Accuracy: ', accuracy_score(Y_test, Y_pred))
#     print('Precision: ', precision_score(Y_test, Y_pred, average='weighted'))
#     print('Recall: ', recall_score(Y_test, Y_pred, average='weighted'))

In [24]:
print('Accuracy: ', accuracy_score(Y_test, Y_pred))
print('Precision: ', precision_score(Y_test, Y_pred, average='weighted'))
print('Recall: ', recall_score(Y_test, Y_pred, average='weighted'))

Accuracy:  0.144652624229
Precision:  0.400912141504
Recall:  0.277450871544


/opt/conda/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [37]:
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.model_selection import train_test_split
from sklearn.multioutput import MultiOutputClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier,AdaBoostClassifier
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import make_scorer, accuracy_score, f1_score, fbeta_score, classification_report
from scipy.stats import hmean
from scipy.stats.mstats import gmean

In [53]:
def tokenize_text(text):
    url_regex = 'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'
    detected_urls = re.findall(url_regex, text)
    for url in detected_urls:
        text = text.replace(url, "urlplaceholder")

    tokens = word_tokenize(text)
    lemmatizer = WordNetLemmatizer()

    clean_tokens = []
    for tok in tokens:
        clean_tok = lemmatizer.lemmatize(tok).lower().strip()
        clean_tokens.append(clean_tok)

    return clean_tokens

In [57]:
class StartingVerbExtractor(BaseEstimator, TransformerMixin):

    def starting_verb(self, text):
        sentence_list = nltk.sent_tokenize(text)
        for sentence in sentence_list:
            pos_tags = nltk.pos_tag(tokenize_text(sentence))
            if len(pos_tags) == 0:
                print('pos_tags:', pos_tags)
            first_word, first_tag = pos_tags[0]
            if first_tag in ['VB', 'VBP'] or first_word == 'RT':
                return True
        return False

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        X_tagged = pd.Series(X).apply(self.starting_verb)
        return pd.DataFrame(X_tagged)

In [58]:
def new_model_pipeline():
    pipeline = Pipeline([
        ('features', FeatureUnion([

            ('text_pipeline', Pipeline([
                ('vect', CountVectorizer(tokenizer=tokenize)),
                ('tfidf', TfidfTransformer())
            ])),

            ('starting_verb', StartingVerbExtractor())
        ])),

        ('clf', MultiOutputClassifier(AdaBoostClassifier()))
    ])

    return pipeline

In [59]:
def multioutput_fscore(y_true,y_pred,beta=1):
    score_list = []
    if isinstance(y_pred, pd.DataFrame) == True:
        y_pred = y_pred.values
    if isinstance(y_true, pd.DataFrame) == True:
        y_true = y_true.values
    for column in range(0,y_true.shape[1]):
        score = fbeta_score(y_true[:,column],y_pred[:,column],beta,average='weighted')
        score_list.append(score)
    f1score_numpy = np.asarray(score_list)
    f1score_numpy = f1score_numpy[f1score_numpy<1]
    f1score = gmean(f1score_numpy)
    return  f1score

In [60]:
model = new_model_pipeline()

parameters = {
    'features__text_pipeline__vect__ngram_range': ((1, 1), (1, 2)),
#    'features__text_pipeline__vect__max_df': (0.75, 1.0),
#    'features__text_pipeline__vect__max_features': (None, 5000),
#    'features__text_pipeline__tfidf__use_idf': (True, False),
#    'clf__n_estimators': [10, 100],
#    'clf__learning_rate': [0.01, 0.1],
#    'features__transformer_weights': (
#        {'text_pipeline': 1, 'starting_verb': 0.5},
#        {'text_pipeline': 0.5, 'starting_verb': 1},
#        {'text_pipeline': 0.8, 'starting_verb': 1},
#    )
}

scorer = make_scorer(multioutput_fscore,greater_is_better = True)

cv = GridSearchCV(model, param_grid=parameters, scoring = scorer,verbose = 2, n_jobs = -1)

cv.fit(X_train, Y_train)

Fitting 5 folds for each of 2 candidates, totalling 10 fits
[CV] features__text_pipeline__vect__ngram_range=(1, 1) ...............
[CV]  features__text_pipeline__vect__ngram_range=(1, 1), total= 1.9min
[CV] features__text_pipeline__vect__ngram_range=(1, 1) ...............


[Parallel(n_jobs=-1)]: Done   1 out of   1 | elapsed:  1.9min remaining:    0.0s


[CV]  features__text_pipeline__vect__ngram_range=(1, 1), total= 1.9min
[CV] features__text_pipeline__vect__ngram_range=(1, 1) ...............
[CV]  features__text_pipeline__vect__ngram_range=(1, 1), total= 1.9min
[CV] features__text_pipeline__vect__ngram_range=(1, 1) ...............
[CV]  features__text_pipeline__vect__ngram_range=(1, 1), total= 1.9min
[CV] features__text_pipeline__vect__ngram_range=(1, 1) ...............
[CV]  features__text_pipeline__vect__ngram_range=(1, 1), total= 1.9min
[CV] features__text_pipeline__vect__ngram_range=(1, 2) ...............
[CV]  features__text_pipeline__vect__ngram_range=(1, 2), total= 3.6min
[CV] features__text_pipeline__vect__ngram_range=(1, 2) ...............
[CV]  features__text_pipeline__vect__ngram_range=(1, 2), total= 3.4min
[CV] features__text_pipeline__vect__ngram_range=(1, 2) ...............
[CV]  features__text_pipeline__vect__ngram_range=(1, 2), total= 3.4min
[CV] features__text_pipeline__vect__ngram_range=(1, 2) ...............
[CV]  

[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed: 26.8min finished


GridSearchCV(cv=None, error_score=nan,
             estimator=Pipeline(memory=None,
                                steps=[('features',
                                        FeatureUnion(n_jobs=None,
                                                     transformer_list=[('text_pipeline',
                                                                        Pipeline(memory=None,
                                                                                 steps=[('vect',
                                                                                         CountVectorizer(analyzer='word',
                                                                                                         binary=False,
                                                                                                         decode_error='strict',
                                                                                                         dtype=<class 'numpy.int64'>,
               

### 9. Export your model as a pickle file

In [61]:
import joblib
joblib.dump(cv.best_estimator_, 'disaster_model.pkl')

['disaster_model.pkl']

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.